In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
df = pd.read_csv("../input/insurance.csv")
df.head(5)
df.tail(10)

In [ ]:
df.info()
df.describe()
df.columns
df.shape
df.smoker.value_counts()
df.isnull().sum()

In [ ]:
#This class assigns a separate number to the 0 to n combination that makes it categorical attributes.
from sklearn.preprocessing import LabelEncoder
#sex
le = LabelEncoder()
le.fit(df.sex.drop_duplicates()) 
df.sex = le.transform(df.sex)
# smoker or not
le.fit(df.smoker.drop_duplicates()) 
df.smoker = le.transform(df.smoker)
#region
le.fit(df.region.drop_duplicates()) 
df.region = le.transform(df.region)

df.head(20)

In [ ]:
Korelasyonu ısı haritası ile gözlemlediğimizde fiyatı en çok etkileyen alanın
sigara içip içmeme durumu olduğunu farkediyoruz. vücut kitle indeksinin de etkisi olmasını beklerdim
ama veri seti içerisinde bu konuda uç veriler bulunmadığı için böyle bir sonuç almış olabileceğimizi
düşünüyorum.

In [ ]:
f, ax = plt.subplots(figsize=(10, 8))
corr = df.corr()
sns.heatmap(corr, mask=np.zeros_like(corr, dtype=np.bool), cmap=sns.diverging_palette(240,10,as_cmap=True),
            square=True, ax=ax)
plt.show()

In [ ]:
df1 = df.loc[:,["age","smoker","bmi","children"]]
df1.plot(subplots=True)
plt.show()

In [ ]:
Veri seti içerisinde sigara içmeme durumunda karşımıza çıkan uç değerler aşağıdaki gibidir.
Bunun nedeni;
yaşanılan bölgeye göre fiyatların değişiyor olması olabilir
ya da çocuk sayısı etkisine de bakmak gerekecektir.

In [ ]:
plt.figure(figsize=(12,5))
plt.title("Box plot for charges")
sns.boxplot(y="smoker", x="charges", data =  df , orient="h", palette = 'magma')
plt.show()

In [ ]:
f= plt.figure(figsize=(12,5))

ax=f.add_subplot(121)
sns.distplot(df[(df.smoker == 1)]["charges"],color='c',ax=ax)
ax.set_title('Sigara içenler için maliyet dağılımı')

ax=f.add_subplot(122)
sns.distplot(df[(df.smoker == 0)]['charges'],color='b',ax=ax)
ax.set_title('Sigara içmeyenler için maliyet dağılımı')

plt.show()

In [ ]:
fig,axes = plt.subplots(nrows=3,ncols=1)
df.plot(kind="hist", y="age", bins=50,range=(0,50),normed=False,ax=axes[0])
df.plot(kind="hist", y="smoker", bins=50,range=(0,50),normed=False,ax=axes[1])
df.plot(kind="hist", y="bmi", bins=50,range=(0,50),normed=False,ax=axes[2])
plt.show()

In [ ]:
sns.catplot(x="smoker", kind="count",hue = 'sex', palette="pink", data=df)
plt.show()

In [ ]:
df.plot(kind='scatter', x='age', y='charges',alpha = 0.5,color = 'red')
plt.xlabel('age')              
plt.ylabel('charges')
plt.title('age -- charges') 
df.plot(kind='scatter', x='bmi', y='charges',alpha = 0.5,color = 'blue')
plt.xlabel('bmi')              
plt.ylabel('charges')
plt.title('bmi -- charges') 
plt.show()

In [ ]:
plt.figure(figsize=(10,6))
ax = sns.scatterplot(x='bmi',y='charges',data=df,palette='magma',hue='smoker')
ax.set_title('Scatter plot of charges and bmi')


In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import r2_score,mean_squared_error
from sklearn.ensemble import RandomForestRegressor

In [ ]:
x = df.drop(['charges'], axis = 1)
y = df.charges

x_train,x_test,y_train,y_test = train_test_split(x,y, random_state = 0)
lr = LinearRegression().fit(x_train,y_train)

y_train_pred = lr.predict(x_train)
y_test_pred = lr.predict(x_test)

print(lr.score(x_test,y_test))

In [ ]:
plt.figure(figsize=(10,6))

plt.scatter(y_train_pred,y_train_pred - y_train,
          c = 'black', marker = 'o', s = 35, alpha = 0.5,
          label = 'Train data')
plt.scatter(y_test_pred,y_test_pred - y_test,
          c = 'c', marker = 'o', s = 35, alpha = 0.7,
          label = 'Test data')
plt.xlabel('Predicted values')
plt.ylabel('Tailings')
plt.legend(loc = 'upper left')
plt.hlines(y = 0, xmin = 0, xmax = 60000, lw = 2, color = 'red')
plt.show()

In [ ]:
X = df.drop(['charges','region'], axis = 1)
Y = df.charges

quad = PolynomialFeatures(degree = 2)
x_quad = quad.fit_transform(X)

X_train,X_test,Y_train,Y_test = train_test_split(x_quad,Y, random_state = 0)

plr = LinearRegression().fit(X_train,Y_train)

Y_train_pred = plr.predict(X_train)
Y_test_pred = plr.predict(X_test)

print(plr.score(X_test,Y_test))

In [ ]:
plt.figure(figsize=(10,6))

plt.scatter(Y_train_pred,Y_train_pred - Y_train,
          c = 'black', marker = 'o', s = 35, alpha = 0.5,
          label = 'Train data')
plt.scatter(Y_test_pred,Y_test_pred - Y_test,
          c = 'c', marker = 'o', s = 35, alpha = 0.7,
          label = 'Test data')
plt.xlabel('Predicted values')
plt.ylabel('Tailings')
plt.legend(loc = 'upper left')
plt.hlines(y = 0, xmin = 0, xmax = 60000, lw = 2, color = 'red')
plt.show()

In [ ]:
forest = RandomForestRegressor(n_estimators = 100,
                              criterion = 'mse',
                              random_state = 1,
                              n_jobs = -1)
forest.fit(x_train,y_train)
forest_train_pred = forest.predict(x_train)
forest_test_pred = forest.predict(x_test)

#print('MSE train data: %.3f, MSE test data: %.3f' % (
#mean_squared_error(y_train,forest_train_pred),
#mean_squared_error(y_test,forest_test_pred)))
#print('R2 train data: %.3f, R2 test data: %.3f' % (
#r2_score(y_train,forest_train_pred),
r2_score(y_test,forest_test_pred)

In [ ]:
plt.figure(figsize=(10,6))

plt.scatter(forest_train_pred,forest_train_pred - y_train,
          c = 'black', marker = 'o', s = 35, alpha = 0.5,
          label = 'Train data')
plt.scatter(forest_test_pred,forest_test_pred - y_test,
          c = 'c', marker = 'o', s = 35, alpha = 0.7,
          label = 'Test data')
plt.xlabel('Predicted values')
plt.ylabel('Tailings')
plt.legend(loc = 'upper left')
plt.hlines(y = 0, xmin = 0, xmax = 60000, lw = 2, color = 'red')
plt.show()